## FINANCIAL DATA
MODULE 6 | LESSON 3


---

# **TECHNICAL ANALYSIS**


|  |  |
|:---|:---|
|**Reading Time** |  35 minutes |
|**Prior Knowledge** | Python  |
|**Keywords** |Trend, Momentum, Average Directional Index (ADX), Moving Average Convergence/Divergence (MACD), <br> Parabolic SAR (Stop and Reverse), Relative Strength Index (RSI), Stochastic Oscillator  |   	

---

*In the last lesson, we analyzed the prices of various cryptocurrencies by finding their correlations. In this lesson, we analyze prices again, but looking for patterns identified by technical analysis.
We discussed technical analysis in the Financial Markets course in Module 6, Lesson 4. In short, "technical analysis is a security analysis discipline for forecasting the direction of prices through the study of past market data, primarily price and volume" ("Technical Analysis" 1). We find that a freely available repository does most of the coding work for us, but we also take a closer look at some of this code so we understand what it is doing and how to modify it if we want it to do something different.*

## 1. The TA Library

As mentioned the TA (technical analysis) repo already provides the code for so much of the technical analysis that we might want to do. [Read more about TA here.](https://github.com/bukosabino/ta)

Install the TA library and download the `datas.csv`, which is just price and volume data. The next few blocks of code calculate each of the 42 indicators supported by the repo. In the following sections, we will take a close look at a few of the indicators related to trend and momentum.

The required reading, "Technical Analysis," 1-8, will give you a good overview of technical analysis, its history, and how it can be used. 
Also required reading are the sections of this text that relate to the indicators discussed below: ADX (Average Directional Index), MACD (Moving Average Convergence/Divergence), Parabolic SAR (Stop And Reverse), RSI (Relative Strength Index), and the Stochastic Oscillator.


In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from ta import add_all_ta_features
from ta.utils import dropna

mpl.style.use("seaborn")

# Load data
df = pd.read_csv("data/ta-test-data.csv.gz", sep=",")

# Clean NaN values
df = dropna(df)

# Add all ta features
df = add_all_ta_features(
    df, open="Open", high="High", low="Low", close="Close", volume="Volume_BTC"
)

In [ ]:
df.head()  # Check to make sure we

## 2. Average Directional Index (ADX)




Average Directional Index (ADI) is a standard trend indicator. Read about it [in the required reading](https://www.mrao.cam.ac.uk/~mph/Technical_Analysis.pdf (102)). Although we have already imported the TA library, find the code for the `ADXIndicator` class below for ease of reference. <br>

In [ ]:
from ta.utils import IndicatorMixin, _ema, _get_min_max


class ADXIndicator(IndicatorMixin):
    """Average Directional Movement Index (ADX)
    The Plus Directional Indicator (+DI) and Minus Directional Indicator (-DI)
    are derived from smoothed averages of these differences, and measure trend
    direction over time. These two indicators are often referred to
    collectively as the Directional Movement Indicator (DMI).
    The Average Directional Index (ADX) is in turn derived from the smoothed
    averages of the difference between +DI and -DI, and measures the strength
    of the trend (regardless of direction) over time.
    Using these three indicators together, chartists can determine both the
    direction and strength of the trend.
    http://stockcharts.com/school/doku.php?id=chart_school:technical_indicators:average_directional_index_adx
    Args:
        high(pandas.Series): dataset 'High' column.
        low(pandas.Series): dataset 'Low' column.
        close(pandas.Series): dataset 'Close' column.
        window(int): n period.
        fillna(bool): if True, fill nan values.
    """

    def __init__(
        self,
        high: pd.Series,
        low: pd.Series,
        close: pd.Series,
        window: int = 14,
        fillna: bool = False,
    ):
        self._high = high
        self._low = low
        self._close = close
        self._window = window
        self._fillna = fillna
        self._run()

    def _run(self):
        if self._window == 0:
            raise ValueError("window may not be 0")

        close_shift = self._close.shift(1)
        pdm = _get_min_max(self._high, close_shift, "max")
        pdn = _get_min_max(self._low, close_shift, "min")
        diff_directional_movement = pdm - pdn

        self._trs_initial = np.zeros(self._window - 1)
        self._trs = np.zeros(len(self._close) - (self._window - 1))
        self._trs[0] = diff_directional_movement.dropna()[
            0 : self._window  # noqa E203
        ].sum()  # noqa E203
        diff_directional_movement = diff_directional_movement.reset_index(drop=True)

        for i in range(1, len(self._trs) - 1):
            self._trs[i] = (
                self._trs[i - 1]
                - (self._trs[i - 1] / float(self._window))
                + diff_directional_movement[self._window + i]
            )

        diff_up = self._high - self._high.shift(1)
        diff_down = self._low.shift(1) - self._low
        pos = abs(((diff_up > diff_down) & (diff_up > 0)) * diff_up)
        neg = abs(((diff_down > diff_up) & (diff_down > 0)) * diff_down)

        self._dip = np.zeros(len(self._close) - (self._window - 1))
        self._dip[0] = pos.dropna()[0 : self._window].sum()  # noqa E203

        pos = pos.reset_index(drop=True)

        for i in range(1, len(self._dip) - 1):
            self._dip[i] = (
                self._dip[i - 1]
                - (self._dip[i - 1] / float(self._window))
                + pos[self._window + i]
            )

        self._din = np.zeros(len(self._close) - (self._window - 1))
        self._din[0] = neg.dropna()[0 : self._window].sum()  # noqa E203

        neg = neg.reset_index(drop=True)

        for i in range(1, len(self._din) - 1):
            self._din[i] = (
                self._din[i - 1]
                - (self._din[i - 1] / float(self._window))
                + neg[self._window + i]
            )

    def adx(self) -> pd.Series:
        """Average Directional Index (ADX)
        Returns:
            pandas.Series: New feature generated.tr
        """
        dip = np.zeros(len(self._trs))

        for idx, value in enumerate(self._trs):
            dip[idx] = 100 * (self._dip[idx] / value)

        din = np.zeros(len(self._trs))

        for idx, value in enumerate(self._trs):
            din[idx] = 100 * (self._din[idx] / value)

        directional_index = 100 * np.abs((dip - din) / (dip + din))

        adx_series = np.zeros(len(self._trs))
        adx_series[self._window] = directional_index[
            0 : self._window  # noqa E203
        ].mean()

        for i in range(self._window + 1, len(adx_series)):
            adx_series[i] = (
                (adx_series[i - 1] * (self._window - 1)) + directional_index[i - 1]
            ) / float(self._window)

        adx_series = np.concatenate((self._trs_initial, adx_series), axis=0)
        adx_series = pd.Series(data=adx_series, index=self._close.index)

        adx_series = self._check_fillna(adx_series, value=20)
        return pd.Series(adx_series, name="adx")

    def adx_pos(self) -> pd.Series:
        """Plus Directional Indicator (+DI)
        Returns:
            pandas.Series: New feature generated.
        """
        dip = np.zeros(len(self._close))
        for i in range(1, len(self._trs) - 1):
            dip[i + self._window] = 100 * (self._dip[i] / self._trs[i])

        adx_pos_series = self._check_fillna(
            pd.Series(dip, index=self._close.index), value=20
        )
        return pd.Series(adx_pos_series, name="adx_pos")

    def adx_neg(self) -> pd.Series:
        """Minus Directional Indicator (-DI)
        Returns:
            pandas.Series: New feature generated.
        """
        din = np.zeros(len(self._close))
        for i in range(1, len(self._trs) - 1):
            din[i + self._window] = 100 * (self._din[i] / self._trs[i])

        adx_neg_series = self._check_fillna(
            pd.Series(din, index=self._close.index), value=20
        )
        return pd.Series(adx_neg_series, name="adx_neg")

Now we'll take a look at a somewhat arbitrary window of the sample data. The code to show the positive and negative ADX are there but commented out so the graph is legible.

In [ ]:
fig, ax1 = plt.subplots()
ax2 = ax1.twinx()
ax1.plot(df[40000:41000].Close, "g-", label="Close")
# ax2.plot(df[40000:41000].trend_adx_pos, 'b-', label='ADX Pos')
# ax2.plot(df[40000:41000].trend_adx_neg, 'r-', label='ADX Neg' )
ax2.plot(df[40000:41000].trend_adx, "k-", label="ADX")

ax1.set_ylabel("Price", color="g")
ax2.set_ylabel("ADX", color="b")

plt.title("ADX and Price", color="k")
plt.legend()
plt.show()

Let's see if the guidelines from the "Technical Analysis" text suggest a winning strategy here: <br>
"Generally, ADX readings below 20 indicate trend weakness, and readings above 40 indicate trend strength. An extremely strong trend is indicated by readings above 50." <br>
As a simplistic assessment of this indicator, take a minute to visually inspect the graph and note whether buying or selling (depending on the direction of the trend at the time) appears to be a profitable strategy. Where does the black ADX line cross, say, 50 (on the right-hand side y-axis) just before a steep increase or decrease in the price (the green line)? Certainly, a strategy that involved buying the stock given the peak ADX (about 75) just after 46200 would have performed well if it was held until ADX declined again to 25 (at about 46250). Similarly, the next time that ADX surpassed 50 (at around 46480) would have been a good time to buy, and when ADX got as low as 25, that would have been a good time to sell. This visual thought experiment is far from scientific, but it should give you a sense for how a trend indicator can be used.


## 3. Moving Average Convergence/Divergence

As you study the required reading about Moving Average Convergence/Divergence (MACD) in "Technical Analysis" (104-108), ensure you understand how to compute and interpret it. <br>
Again, the code from the TA library for the MACD class is provided below for your reference. 



In [ ]:
class MACD(IndicatorMixin):
    """Moving Average Convergence Divergence (MACD)
    Is a trend-following momentum indicator that shows the relationship between
    two moving averages of prices.
    Args:
        close(pandas.Series): dataset 'Close' column.
        window_fast(int): n period short-term.
        window_slow(int): n period long-term.
        window_sign(int): n period to signal.
        fillna(bool): if True, fill nan values.
    """

    def __init__(
        self,
        close: pd.Series,
        window_slow: int = 26,
        window_fast: int = 12,
        window_sign: int = 9,
        fillna: bool = False,
    ):
        self._close = close
        self._window_slow = window_slow
        self._window_fast = window_fast
        self._window_sign = window_sign
        self._fillna = fillna
        self._run()

    def _run(self):
        self._emafast = _ema(self._close, self._window_fast, self._fillna)
        self._emaslow = _ema(self._close, self._window_slow, self._fillna)
        self._macd = self._emafast - self._emaslow
        self._macd_signal = _ema(self._macd, self._window_sign, self._fillna)
        self._macd_diff = self._macd - self._macd_signal

    def macd(self) -> pd.Series:
        """MACD Line
        Returns:
            pandas.Series: New feature generated.
        """
        macd_series = self._check_fillna(self._macd, value=0)
        return pd.Series(
            macd_series, name=f"MACD_{self._window_fast}_{self._window_slow}"
        )

    def macd_signal(self) -> pd.Series:
        """Signal Line
        Returns:
            pandas.Series: New feature generated.
        """

        macd_signal_series = self._check_fillna(self._macd_signal, value=0)
        return pd.Series(
            macd_signal_series,
            name=f"MACD_sign_{self._window_fast}_{self._window_slow}",
        )

    def macd_diff(self) -> pd.Series:
        """MACD Histogram
        Returns:
            pandas.Series: New feature generated.
        """
        macd_diff_series = self._check_fillna(self._macd_diff, value=0)
        return pd.Series(
            macd_diff_series, name=f"MACD_diff_{self._window_fast}_{self._window_slow}"
        )

In [ ]:
fig, ax1 = plt.subplots()
ax2 = ax1.twinx()
ax1.plot(df[40640:42000].Close, "g-", label="Close")
# ax2.plot(df[40640:42000].trend_macd, 'b-', label='MACD')
# ax2.plot(df[40640:42000].trend_macd_signal, 'r-', label='MACD Signal' )
ax2.plot(df[40640:42000].trend_macd_diff, "k-", label="MACD Difference")

ax1.set_ylabel("Price", color="g")
ax2.set_ylabel("MACD Levels", color="b")

ax2.set_ylim(-20, 20)
ax2.axhline(0)

plt.title("Price + MACD, MACD Signal and MACD Difference", color="k")
plt.legend()
plt.show()



As you read through the code above, make sure you understand how the code works and how to use the built-in functions. For example, [see this required reading to ensure you are familiar with the built-in function "enumerate."](https://www.w3resource.com/python/built-in-function/enumerate.php)



## 4. Other Trend and Momentum Indicators


[Watch this required reading video about parabolic SAR (which we discussed in passing in Financial Markets)](https://www.youtube.com/watch?v=kFTcvDsU1ik)

[Watch this required reading video about how all three of these trend indicators (ADX, MACD, and parabolic SAR) can be used together](https://www.youtube.com/watch?v=hCo9ax0W2Bw)

[Watch this required reading video about relative strength index (RSI)](https://www.youtube.com/watch?v=01spTWPsRNY)

[Watch this required reading video about the stochastic oscillator](https://www.youtube.com/watch?v=bb9ms3fUK7k)

## 5. Conclusion

We just used some interesting functions to calculate different metrics that might contain information about when to buy or sell a security. These indicators were based exclusively on current and previous prices--without any other information about the stock or the company. In the next lesson, we do take into account data that is specific to the companies we are investigating. This is generally financial data that is available from the financial statements, in addition to the current stock price. That is, we are moving from technical analysis to fundamental analysis.

**References**

* eVidhya. "ADX MACD Parabolic SAR strategy | Forex strategies." https://www.youtube.com/watch?v=hCo9ax0W2Bw

* ForexBoat. "Stochastic Oscillator Settings & Trading Strategy in Forex." https://www.youtube.com/watch?v=bb9ms3fUK7k

* ForexBoat. "Parabolic SAR Indicator Formula and Strategy." https://www.youtube.com/watch?v=kFTcvDsU1ik

* Padial, Darío López. "Technical Analysis Library in Python." *GitHub*. https://github.com/bukosabino/ta

* Stas Serfes. "How to Use the RSI Indicator on ThinkOrSwim (Relative Strength Index)." https://www.youtube.com/watch?v=01spTWPsRNY

* *Technical Analysis*, PDF book, 2011, Creative Commons Attribution-Share, available from: https://www.mrao.cam.ac.uk/~mph/Technical_Analysis.pdf

* W3resource. "Python: enumerate() function" https://www.w3resource.com/python/built-in-function/enumerate.php

---
Copyright © 2022 WorldQuant University. This
content is licensed solely for personal use. Redistribution or
publication of this material is strictly prohibited.
